In [1]:
# always import
import sys
from time import time

# numpy & scipy
import numpy as np
import scipy
import random

# sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import pairwise_distances_argmin, pairwise_distances
from sklearn import neighbors
from scipy.spatial.distance import pdist,squareform

# Hungarian algorithm
from munkres import Munkres
from scipy.optimize import linear_sum_assignment

# visuals
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn.manifold import Isomap, TSNE

# maybe
from numba import jit

In [2]:
# load MNIST data and normalization
from sklearn.datasets import fetch_openml
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, data_home='mnist/')
y = np.asarray(list(map(int, y)))
X = np.asarray(X.astype(float))
X = scale(X)
n_digits = len(np.unique(y))

# print("X shape", X.shape)

In [3]:
# TODO: Kmeans

def Kmeans(X, k, iterations, threshold, initial_cluster_centers):
    '''
    Input:
    X - input vector 
    k - number of clusters 
    iterations - maximum number of iterations of the Kmeans algorithm
    threshold - termination criterion
    calculated_cluster_centers - inital cluster centers calculated 
    
    Returns:
    cluster_centers - k x m matrix with updated cluster centers
    closest_cl - n x 1 vector with the output labels 
    cost_final  
    '''
    i = 0
    # Creating a copy of the initial cluster_centers
    cluster_centers = initial_cluster_centers
    
    # point cluster stores the closest cluster to each data point 
    closest_cl = np.zeros((X.shape[0]))
    closest_cl_vec = np.zeros((X.shape))
    cost_diff = 0
    
    while i<iterations:
        
        cost_init = np.sum(np.square((X - closest_cl_vec)))
#         print("initial cost", cost_init)
        
        #Assigning each data point to the closest centroid
        for j in range(X.shape[0]):
            closest_cl[j] = closest_cluster_index(X[j,:], cluster_centers)
            closest_cl_vec[j,:] = cluster_centers[int(closest_cl[j]),:] 
        
        # Updating Cluster centroids as mean of data points assigned to the cluster
        for l in range(k):
            args_l = np.argwhere(closest_cl == l).flatten()
#             print("args_l", args_l)
            cluster_centers[l,:] = np.mean(X[args_l,:], axis=0)
        
#         print("clus center -", cluster_centers)       
        
        cost_final = np.sum(np.square((X - closest_cl_vec)))
        cost_diff = cost_init - cost_final
        
        if(threshold > cost_diff):
            print("threshold reached")
            
            break
    
        i+=1
        
    # returning the output labels and resulting centroids
    print("No. Iterations - ", i)
    print("cost final", cost_final)
#     print("Shape of output label - ", closest_cl.shape)
    return cluster_centers, closest_cl, cost_final   

def closest_cluster_index(point, cluster_centers):
    '''
    Function to find the index of the cluster centroid which is closest to the given point 
    
    point - 1 x m array representing a point from which distance is to be calculate
    cluster_centers - k x m array containing the k cluster centers 
    
    '''
    #Using euclidean distance
    dist = np.linalg.norm((cluster_centers-point),axis=1)
    closest_centroid_ind = np.argmin(dist)
    return closest_centroid_ind
        

def initialise_clustercentroids(X, K, method, seed=100):
    '''
    Function to initialise the cluster centers either by using PCA or random sampling from X
    '''
    #Choosing top k components of PCA
    if(method == 'pca'):
        pca = PCA(n_components = K)
        pca.fit(X)
        cluster_centers = pca.components_
        #centers is k x m where m is 784
        return cluster_centers
    
    if(method == 'random'):
        np.random.seed(seed)
        ind = np.random.choice(X.shape[0], K, replace= False)
        centers = X[ind,:]
        return centers

In [4]:
#Samples represents number of samples of X we want to use for execution
# Clustering - I
samples = 70000

threshold = 1e-4
k = 10
method = 'pca'

#initialise clusters
cl_centers = initialise_clustercentroids(X, k, method)

#call kmeans
cluster, op_label, final_cost = Kmeans(X[:samples,:], k, 300, threshold, cl_centers)

print("cluster", cluster)

threshold reached
No. Iterations -  121
cost final 42569808.322074406
cluster [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [18]:
# TODO: Hungarian algorithm
# 3.2

def hungarian_costmatrix(yl, zl, K):
    W = np.zeros((K,K))
    for i in range(K):
        set_zl =  np.argwhere(zl == i).flatten()
        for j in range(K):            
            set_yl = np.argwhere(yl == j).flatten()
            W[i,j] = 1 - np.size(np.intersect1d(set_zl,set_yl ))

    min_W = W.min(axis=1)
    W = (W.transpose() - min_W).transpose()
    return W

def accuracy_metrics(cm, row, col):
    
    true_pos = np.diag(cm)
    false_pos = np.sum(cm, axis=0) - true_pos
    false_neg = np.sum(cm, axis=1) - true_pos

    precision = np.sum(true_pos / true_pos+false_pos)
    recall = np.sum(true_pos / true_pos + false_neg)
    accuracy = (row==col).sum()/10
    print(" Confusion Matrix - \n{}\nAccuracy - {} ".format(cm, accuracy))
    return None

#3.1.1
#Similarity scores
def similarity_scores(y, op_label):
    print("Calculating the similarity scores for the predicted labels of KMeans : \n")
    #Homogeneity
    h = metrics.homogeneity_score(y, op_label)
    #completeness_score
    cs = metrics.completeness_score(y, op_label)
    #v_measure_score
    vm = metrics.v_measure_score(y, op_label)
    #adjusted_rand_score
    ars = metrics.adjusted_rand_score(y, op_label)
    #adjusted_mutual_info_score
    amis = metrics.adjusted_mutual_info_score(y, op_label)
    print(" Homogeneity score - {} \n Completeness Score - {} \n V measure - {} \n Adjusted Random Score - {} \n Adjusted mutual info score - {} \n ".format(h, cs, vm, ars, amis))
 

W_I = hungarian_costmatrix(y[:samples], op_label, 10)
print(op_label.shape)
row_I, col_I = linear_sum_assignment(W_I)

cm_I = confusion_matrix(row_I, col_I)

accuracy_metrics(cm_I, row_I, col_I)
similarity_scores(y[:samples], op_label) 

(70000,)
 Confusion Matrix - 
[[0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]]
Accuracy - 0.1 
Calculating the similarity scores for the predicted labels of KMeans : 

 Homogeneity score - 0.2776736500859776 
 Completeness Score - 0.3088967245868502 
 V measure - 0.29245418697961034 
 Adjusted Random Score - 0.19065855092748982 
 Adjusted mutual info score - 0.27749188734624725 
 


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide
D:\Anaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [11]:
#3.1.2 - PCA dimensionality reduction - setting n_components = 30
# Run Kmeans 10 times on the X_ with different random initializaiton of cluster centroid
# Find run with lowest objective value
# Clustering - II

pca = PCA(n_components = 30)
pca.fit(X[:samples,:])
X_ = pca.transform(X[:samples,:])

lowest_cost = sys.maxsize
method = 'random'
for i in range(10):
    seed = random.randint(1,1000)
    cl_centers = initialise_clustercentroids(X_, k, method, seed)
    cluster, op_label, final_cost = Kmeans(X_, k, 300, threshold, cl_centers)
    
    if(final_cost < lowest_cost):
        lowest_cost = final_cost
        best_cluster = cluster
        best_op_label = op_label
    
similarity_scores(y[:samples], best_op_label)    


threshold reached
No. Iterations -  0
cost final 22932729.971414596
threshold reached
No. Iterations -  0
cost final 23143082.283028025
threshold reached
No. Iterations -  0
cost final 23462810.89302005
threshold reached
No. Iterations -  0
cost final 23444974.52218463
threshold reached
No. Iterations -  167
cost final 15079341.188106926
threshold reached
No. Iterations -  0
cost final 23235309.493497517
threshold reached
No. Iterations -  90
cost final 15016728.40848851
threshold reached
No. Iterations -  0
cost final 25479154.84332352
threshold reached
No. Iterations -  0
cost final 25947693.976061404
threshold reached
No. Iterations -  0
cost final 23192994.764118455
Calculating the similarity scores for the predicted labels of KMeans using PCA for initial clusters : 

 Homogeneity score - 0.417931151199491 
 Completeness Score - 0.4399201639162582 
 V measure - 0.42864383909368486 
 Adjusted Random Score - 0.31900912808579157 
 Adjusted mutual info score - 0.4177847086797904 
 


D:\Anaconda\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)


In [20]:

# Apply hungarian algorithm to the cost matricies obtained for the two clusterings 
W_I = hungarian_costmatrix(y[:samples], op_label, 10)
W_II = hungarian_costmatrix(y[:samples], best_op_label, 10)

print("Cost matrix W_I is \n",W_I)

print("Cost matrix W_II is \n",W_II)

#Call linear_sum_assignment to compute the hundarian algorithm
row_I, col_I = linear_sum_assignment(W_I)
print("Achieved assignments - \n row:{}\n col:{}".format(row_I,col_I))

row_II, col_II = linear_sum_assignment(W_II)
print("Achieved assignments - \n row:{}\n col:{}".format(row_II,col_II))

# Obtain confusion matrix from the row and column indices

cm_I = confusion_matrix(row_I, col_I)
cm_II = confusion_matrix(row_II, col_II)

#Find accuracy metrics 
accuracy_metrics(cm_I, row_I, col_I)
accuracy_metrics(cm_II, row_II, col_II)

Cost matrix W_I is 
 [[4038.    0. 3800. 2970. 3036. 2934. 3927. 3794. 3541. 3216.]
 [2896. 3413. 2487. 2385. 2774. 1862. 3431. 3165.    0. 2852.]
 [5548. 5667. 4785. 4701. 2477. 5375. 5507.    0. 5422. 2098.]
 [1061. 1288. 1023. 1281. 1268. 1283.    0. 1289. 1286. 1283.]
 [ 873.  897.  844.  895.  843.  887.    0.  897.  888.  884.]
 [3101. 3256. 2526. 3243. 2891. 3192.    0. 3262. 3255. 3253.]
 [3407.    0. 2425. 3308. 3222. 3071. 3174. 3163. 2790. 3307.]
 [1952. 1986.    0. 1677. 1876. 1877. 1982. 1899. 1323. 1112.]
 [ 707. 1105. 1050.    0.  944.  553. 1082. 1069.  792. 1071.]
 [   0. 4997. 4556. 2885. 4331. 3139. 4507. 4655. 4364. 4452.]]
Cost matrix W_II is 
 [[2797. 4119. 3397.    0. 4148. 2004. 4051. 4138. 1567. 4033.]
 [4917. 5311. 4433. 5243. 5166. 5225.    0. 5325. 5289. 5322.]
 [1305. 1776.    0.  679. 1758. 1631. 1775. 1769. 1709. 1771.]
 [2117. 2535. 2627. 2605. 1805.    0. 2567. 2576.  665. 2588.]
 [7588.    0. 6791. 7056. 7081. 7148. 7061. 7088. 6437. 7263.]
 [3837. 384

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


In [21]:
# TODO: Spectral clustering
#3.3 Spectral Clustering with Kmeans

H = neighbors.NearestNeighbors(n_neighbors=500,algorithm='kd_tree',metric='euclidean').fit(X_).kneighbors_graph(mode='distance')
sigma = (1/H.count_nonzero())*np.sum(H.data)
E = H.copy()
E.data = np.exp(-np.square(np.divide(H.data,sigma)))

#Normalizing E
E.setdiag(0)
E = E/E.sum()
E = (E + E.transpose())/2

#Forming the D matrix

d = np.reciprocal(np.sqrt(E.toarray().sum(axis=1)))
D = scipy.sparse.csr_matrix((E.shape))
D.setdiag(d)


I = scipy.sparse.csr_matrix(np.eye(E.shape[0]))

print("Shapes of E, D, I", E.shape, D.shape, I.shape)


L = scipy.sparse.csr_matrix( I - D.dot(E.dot(D)) )

print("L found")
v,w = scipy.sparse.linalg.eigs(L,20)
print("Eigen values and vectors found")
v=np.real(v)
w = np.real(w)
# print(v.shape)


sorted_eigVecs =  w[:,np.argsort(v)]
k_eigVecs = w[:,1:k]
print(k_eigVecs.shape)

km = KMeans(init='k-means++', n_clusters=10, n_init=k)
km.fit(k_eigVecs)

spec_op_labels = km.labels_
W_III = hungarian_costmatrix(y[:samples], spec_op_labels, k)
row_III, col_III = linear_sum_assignment(W_III)

cm_III = confusion_matrix(row_III, col_III)

accuracy_metrics(cm_III, row_III, col_III)


D:\Anaconda\lib\site-packages\scipy\sparse\compressed.py:708: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self[i, j] = values


MemoryError: 

In [ ]:
# TODO: Clustering + KNN
#3.4
